# Who owns the Fish?

This logic puzzle is attributed to Einstein, who claimed that 98\% of
world could not figure it out.  We are way smarter than Einstein
because we know integer programming.  Write an integer program that
will solve the following problem.

In a street there are five houses, painted five different colours.
In each house lives a person of different nationality.
These five homeowners each drink a different kind of beverage, smoke
different brand of cigar and keep a different pet.

### Who owns the fish?

Hints:
- The Brit lives in a red house.
- The Swede keeps dogs as pets.
- The Dane drinks tea.
- The Green house is on the left of the White house. (i.e. the number
    of the white house is  one greater than the green house)
- The owner of the Green house drinks coffee.
- The person who smokes Pall Mall rears birds.
- The owner of the Yellow house smokes Dunhill.
- The man living in the centre house drinks milk.
- The Norwegian lives in the first house.
- The man who smokes Blends lives next to the one who keeps cats.
- The man who keeps horses lives next to the man who smokes Dunhill.
- The man who smokes Blue Master drinks beer.
- The German smokes Prince.
- The Norwegian lives next to the blue house.
- The man who smokes Blends has a neighbour who drinks water.

In [2]:
# Load the gams extension
%load_ext gams_magic

In [13]:
%%gams
set nationality /Brit, German, Norwegian, Dane, Swede/
    smoke /Blends, Dunhill, BlueMaster, PallMall, Prince/
    beverage/tea, beer, milk, water, coffee/
    color /yellow, red, green, white, blue/
    pet /dog, bird, cat, horse, fish/;
*set fishowner(nationality);

set i 'house number' /1*5/;
parameter house /1 1, 2 2, 3 3, 4 4, 5 5/;

binary variable N(nationality,i) 'Nationality of the house';
equations Neq1, Neq2;
Neq1(i).. sum(nationality, N(nationality,i)) =E= 1;
Neq2(nationality).. sum(i, N(nationality,i)) =E= 1;

binary variable H(color,i) 'Color of the house';
equations Heq1, Heq2;
Heq1(i).. sum(color, H(color,i)) =E= 1;
Heq2(color).. sum(i, H(color,i)) =E= 1;

binary variable D(beverage,i) 'Beverage of the house';
equations Deq1, Deq2;
Deq1(i).. sum(beverage, D(beverage,i)) =E= 1;
Deq2(beverage).. sum(i, D(beverage,i)) =E= 1;

binary variable P(pet,i) 'Pet of the house';
equations Peq1, Peq2;
Peq1(i).. sum(pet, P(pet,i)) =E= 1;
Peq2(pet).. sum(i, P(pet,i)) =E= 1;

binary variable C(smoke,i) 'Pet of the house';
equations Ceq1, Ceq2;
Ceq1(i).. sum(smoke, C(smoke,i)) =E= 1;
Ceq2(smoke).. sum(i, C(smoke,i)) =E= 1;

free variable obj;

binary variable b1;
binary variable b2;
binary variable b3;
binary variable b4;

* The Norwegian lives in the first house
N.fx('Norwegian','1') = 1;

* The person living in the middle house drinks milk
D.fx('milk','3') = 1

* The person living in the yellow house smokes Dunhill
equation eq3;
eq3.. sum(i, house(i)*H('yellow',i)) =E= sum(i, house(i)* C('Dunhill',i) );

* The person living in green house drinks coffee
equation eq4;
eq4.. sum(i, house(i)*H('green',i)) =E= sum(i, house(i)* D('coffee',i) );

* The Dane drinks tea
equation eq5;
eq5.. sum(i, house(i)*N('Dane',i)) =E= sum(i, house(i)* D('tea',i) );

* The German smokes Prince
equation eq6;
eq6.. sum(i, house(i)*N('German',i)) =E= sum(i, house(i)* C('Prince',i) );

* The Swede has a dog
equation eq7;
eq7.. sum(i, house(i)*N('Swede',i)) =E= sum(i, house(i)* P('dog',i) );

* The beer drinker smokes BlueMaster
equation eq8;
eq8.. sum(i, house(i)*D('beer',i)) =E= sum(i, house(i)* C('BlueMaster',i) );

* The bird owner smokes Pall Mall
equation eq9;
eq9.. sum(i, house(i)*P('bird',i)) =E= sum(i, house(i)* C('PallMall',i) );

* The green house is situated immediately to the left of the white house
equation eq10;
eq10.. sum(i, house(i)*H('white',i)) - sum(i, house(i)*H('green',i))  =E= 1 ;

* The British person lives in the red house
equation eq11;
eq11.. sum(i, house(i)*N('Brit',i)) =E= sum(i, house(i)* H('red',i) );

* the person who smokes Blend lives next to the owner of cats, there are two possibilities
equation eq12;
eq12.. sum(i, house(i)*C('Blends',i)) - sum(i, house(i)*P('cat',i))  =E= 1 -(2*b1);

* The person who keeps horses lives next to the person who smokes Dunhill
equation eq13;
eq13.. sum(i, house(i)*P('horse',i)) - sum(i, house(i)*C('Dunhill',i))  =E= 1 -(2*b2);

* The person who smokes Blend has a neighbour who drinks water
equation eq14;
eq14.. sum(i, house(i)*C('Blends',i)) - sum(i, house(i)*D('water',i))  =E= 1 -(2*b3);

* Norwegian lives next to the blue house
equation eq15;
eq15.. sum(i, house(i)*N('Norwegian',i)) - sum(i, house(i)*H('blue',i))  =E= 1 -(2*b4);

equation defobj;
defobj.. obj =E= sum((nationality,i), N(nationality,i));

model einstein /all/;
solve einstein using MIP max obj;
display N.L, H.L, P.L;

set fishHome(i);
set fishOwner(nationality, i);

fishHome(i) = yes$(P.L('fish',i) = 1);
fishowner(nationality, fishHome) = yes$(N.l(nationality, fishHome) = 1);

display fishowner;

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),5.0,64,130,MIP,CPLEX,0.015


In [14]:
%gams_pull fishowner
display(fishowner)
print("The " + fishowner[0][0] + " owns the fish")

[['German', '4']]

The German owns the fish


Now check that the solution of this problem is unique.

Hint: Maybe show that additionally enforcing that the __nationality you got above__ does NOT own the fish gives an infeasible problem.

In [16]:
%%gams
P.fx('fish','4') = 0;
solve einstein using mip maximizing obj;

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),IntegerInfeasible (10),NA,64,130,MIP,CPLEX,0.031
